# Follow the Carrot Path Tracking

## Introduction

Previously, I studied [simple PID path tracking and tuning the PIDs using genetic algorithm](1-simple-pid-path-tracking.ipynb). [Follow the carrot](http://www8.cs.umu.se/kurser/TDBD17/VT06/utdelat/Assignment%20Papers/Path%20Tracking%20for%20a%20Miniature%20Robot.pdf) is the second path tracking algorithm I am going to cover in this notebook. This algorithm calls for more involved maths. First we are going to go through the required maths, then implement python classes to plot simulation graphs, and finally implement an animation of the algorithm.

We are going to implement a new class `PathDescriber` instead of the old `DistanceMeter` class implemented for [simple PID path tracking](1-simple-pid-path-tracking.ipynb). `PathDescriber` is the primary class for the new steering control algorithm. Other classes remain pretty much the same as they were in  [simple PID path tracking](1-simple-pid-path-tracking.ipynb).

## The Maths

<img src="img/2/the_maths.png", width=550, height=440>

$\delta$ is the steering angle. $O$ is the origin of the global coordinate system. $P$ is the position of the robot. $P_1$, $P_3$, and $P_5$ are the waypoints. $P_2$ is the center of the circle. $\overline{PP_2}$ is perpendicular to $\overline{P_1P_3}$. $\overline{P_2P_4}$ is defined as **look-ahead distance**. Angle $\psi$ describes the robot orientation with respect to the global $x$ axis. $\alpha_m$ is the measured angle between  the look-ahead distance and the robot orientation.

As the robot moves along the path, we are to find points $P_2$ and $P_4$, then calculate $\alpha_m$. Point $P_4$ can be on the nearest path segment or on any segment after the nearest. It depends on the chosen look-ahead distance. We should iterate the segments from the nearest to the last to locate $P_4$, which is the intersection between a segment on the path and the circle centered at $P_2$ with the radius of $\overline{P_2P_4}$ (i.e. the look-ahead distance).

We can find the center of the circle $P_2$ using the vector $\vec{OP_2}$.

$$\begin{align}
\vec{OP_2} &= \vec{OP_1} + \vec{P_1P_2} \\
\vec{OP_2} &= \vec{OP_1} + \vec{P_1P}\cdot\vec{P_1P_3}
\end{align}$$

$P_4$ can be located using the inersection between the line and the circle. See [Circle-Line Intersection](http://mathworld.wolfram.com/Circle-LineIntersection.html) for more details.

Let $P_k$ be the point $(x_k, y_k)$

Let $(x_k', y_k')$ be  $(x_k - x_2, y_k - y_2)$ (i.e. translate the center of the circle to the origin $O(0,0)$)

$$\begin{align}
d_x &= x_5' - x_3' \\
d_y &= y_5' - y_3' \\
\lvert P_3P_5 \rvert^2 &= d_{x}^2  + d_{y}^2 \\
D &= \begin{vmatrix}
x_3' & x_5'\\
y_3' & y_5'
\end{vmatrix} = x_3'y_5' - x_5'y_3' \\
\text{sgn*}(x) &= \begin{cases}
 -1, & \text{for } x < 0 \\
  1, & \text{otherwise}
\end{cases} \\
\Delta &= \lvert P_2P_4 \rvert ^2 \lvert P_3P_5 \rvert ^2 - D^2 \\
x_4 &= \frac{Dd_y \pm \text{sgn*}(d_y)d_x\sqrt{\Delta}}{\lvert P_3P_5 \rvert^2} + x_2 \\
y_4 &= \frac{Dd_x \pm \lvert d_y \rvert d_x\sqrt{\Delta}}{\lvert P_3P_5 \rvert^2} + y_2
\end{align}$$

| $\Delta$      |  incidence    |
| ------------- |:-------------:|
| $\Delta$ < 0  | no interestion|
| $\Delta$ = 0  | tangent       |
| $\Delta$ > 0  | intersection  |

## Steering Control

![steering_control](img/2/steering_control.png)
As mentioned before, $\alpha_m$ is the measured angle between the look-ahead distance and the robot orientation. $\alpha_d$ is the desired angle and it is always zero, as we want $\alpha_m$ to be zero. Error is given as $\alpha_e = \alpha_d - \alpha_m$. Steering angle $\delta$ is calcuated as follows using a PID controller.

$$\begin{align}
\delta(t) = K_p\alpha_e(t) + K_i\int_{0}^{t}\alpha_e(\tau)\text(d)\tau + K_d\frac{d\alpha_e(t)}{dt}
\end{align}$$

In most cases proportional coefficient $K_p$ is enough to get a satisfactory results. If we need further adjustment, we can set integral and derivative coefficients $K_i$, $K_d$ to nonzero values and tweak them in order to improve the performance. Also note that the steering angle $\delta$ should be mapped to servo signals in practice.